# LLM API Chinese Test
針對市面上常見的 LLM API 進行中文測試，包含 Google, OpenAI, AWS, Mistral, GooseAPI 等，並比較其效果與價格


In [72]:
! pip3 install openai mdprint mistralai

In [73]:
! pip3 install --upgrade google-cloud-aiplatform

In [74]:
from openai import OpenAI
import vertexai
from mistralai.client import MistralClient

OPENAI_KEY = ""
open_ai_client = OpenAI(
  api_key=OPENAI_KEY,
)

# gcloud auth application-default print-access-token
GCP_ACCESS_TOKEN = ""
GOOGLE_PROJECT_ID = ""
vertexai.init(project=GOOGLE_PROJECT_ID)

MISTRAL_AI_KEY = ""
mistral_client = MistralClient(api_key=MISTRAL_AI_KEY)


In [81]:
from openai import OpenAI
from mdprint import mdprint
from vertexai.language_models import TextGenerationModel
from vertexai.generative_models import GenerativeModel
from mistralai.models.chat_completion import ChatMessage

def execute_llm(prompt):
    funcs = [
        OpenAI_API, 
        Gemini_API, 
        Mistral_API
    ]
    for func in funcs:
        print(func.__name__)
        func(prompt)
        print("-------------------")

# OpenAI
def OpenAI_API(prompt):
    models = [
        "gpt-3.5-turbo",
        "gpt-4-turbo-preview",
    ]
    input_token_fees = [
        0.5,
        10,
    ]
    output_token_fees = [
        1.5,
        30
    ]
    for idx, model in enumerate(models):
        print(model)
        response = open_ai_client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ]
        )
        mdprint(response.choices[0].message.content.strip())
        fees = response.usage.prompt_tokens * input_token_fees[idx] + response.usage.completion_tokens * output_token_fees[idx]
        print("Fees (1M calls): ", fees)
        print("Tokens: ", response.usage)


# Google
def Gemini_API(prompt: str):
    multimodal_model = GenerativeModel("gemini-1.0-pro")
    response = multimodal_model.generate_content(
        [
            prompt
        ]
    )
    mdprint(response.text)
    fees = response._raw_response.usage_metadata.prompt_token_count * 0.125 + response._raw_response.usage_metadata.candidates_token_count * 0.375
    print("Fees (1M calls):", fees)
    print("Tokens:", response._raw_response.usage_metadata)

def Mistral_API(prompt: str) -> str:
    models = [
        "open-mistral-7b", # tiny
        "open-mixtral-8x7b",
        "mistral-small-latest", # small
        "mistral-large-latest"
    ]
    input_token_fees = [
        0.25,
        0.7,
        2,
        8
    ]
    output_token_fees = [
        0.25,
        0.7,
        6,
        24
    ]
    for idx, model in enumerate(models):
        print(model)
        messages = [
            ChatMessage(role="user", content=prompt)
        ]

        chat_response = mistral_client.chat(
            model=model,
            messages=messages,
        )

        print(chat_response.choices[0].message.content)
        fees = chat_response.usage.prompt_tokens * input_token_fees[idx] + chat_response.usage.completion_tokens * output_token_fees[idx]
        print("Fees (1M calls):", fees)
        print("Tokens:", chat_response.usage)

# Mistral

In [82]:
def show_llm_result(prompt, contents):
    for content in contents:
        final_prompt = prompt.format(content)
        print('Prompt: {}'.format(final_prompt))
        execute_llm(final_prompt)
        print("=====================================")

In [83]:
# Summary
prompt = """
將在「」中的文字做簡短的中文總結，總結不可超過 20 字，並擷取出內容中用戶給予物流部門的反饋

「{}」
"""

contents = [
    """
    我在女兒生日時買了這隻熊貓布偶，她很喜歡並帶著它到處走。它柔軟可愛，長得可愛，是個不錯的禮物
    但對於價格來說有點小。我認為其他同價位有更好的選擇。運送方面它比預期早了一天送到，所以我在給女兒之前還有時間可以多確認一下。
    """
]

show_llm_result(prompt, contents)


Prompt: 
將在「」中的文字做簡短的中文總結，總結不可超過 20 字，並擷取出內容中用戶給予物流部門的反饋

「
    我在女兒生日時買了這隻熊貓布偶，她很喜歡並帶著它到處走。它柔軟可愛，長得可愛，是個不錯的禮物
    但對於價格來說有點小。我認為其他同價位有更好的選擇。運送方面它比預期早了一天送到，所以我在給女兒之前還有時間可以多確認一下。
    」

OpenAI_API
gpt-3.5-turbo
熊貓布偶受歡迎，但價格稍貴。物流部門提前送達讓用戶有更多時間確認。
Fees (1M calls):  192.0
Tokens:  CompletionUsage(completion_tokens=47, prompt_tokens=243, total_tokens=290)
gpt-4-turbo-preview
總結：購買熊貓布偶為女兒生日禮物，物流快速。

物流反饋：比預期早一天送到。
Fees (1M calls):  4050
Tokens:  CompletionUsage(completion_tokens=54, prompt_tokens=243, total_tokens=297)
-------------------
Gemini_API
布偶品質好，價格偏高，物流比預期快。
Fees (1M calls): 20.625
Tokens: prompt_token_count: 120
candidates_token_count: 15
total_token_count: 135

-------------------
Mistral_API
open-mistral-7b
雖然這只熊貓布偶柔和可愛，但客人認為價格過高，運送快於預期。
Fees (1M calls): 64.25
Tokens: prompt_tokens=214 total_tokens=257 completion_tokens=43
open-mixtral-8x7b
買的熊貓布偶柔軟可愛，早送達；但價格有點高。

使用者對物流部門的反饋：早一天送到了商品。
Fees (1M calls): 193.2
Tokens: prompt_tokens=214 total_tokens=276 completion_tokens=62
mistral-

In [78]:
# Inference
prompt = """
針對「」中的文字進行分析
1. 情緒分析，如果是正面回覆 positive，負面則回覆 negative
2. 針對物品本身，條列式列出優點
3. 根據內容分類是屬於以下哪類商品：拉力帶、腰帶、護膝、其他

「{}」
"""

contents = [
    """
    非常感謝賣家推薦這款拉力帶，作工細緻，無異味，使用起來也很舒適。還有超讚的出貨速度，讓我能儘早拿到購買商品。
    """,
    """
    直上了傳說中最頂級的健身腰帶，質感爆棚做工精細，皮帶超厚給予核心收緊的效果很誇張，果然是比賽也在用的
    """,
    """
    這款護肘質量差，不舒適，戴上後容易滑動，且無法有效減輕肘部問題。
    """,
]

show_llm_result(prompt, contents)


Prompt: 
針對「」中的文字進行分析
1. 情緒分析，如果是正面回覆 positive，負面則回覆 negative
2. 針對物品本身，條列式列出優點
3. 根據內容分類是屬於以下哪類商品：拉力帶、腰帶、護膝、其他

「
    非常感謝賣家推薦這款拉力帶，作工細緻，無異味，使用起來也很舒適。還有超讚的出貨速度，讓我能儘早拿到購買商品。
    」

OpenAI_API
gpt-3.5-turbo
1. 正面回覆
2. 拉力帶的優點：作工細緻、無異味、舒適使用、出貨速度快
3. 屬於拉力帶商品
Fees (1M calls):  197.0
Tokens:  270
gpt-4-turbo-preview
1. 正面回覆：positive

2. 商品優點：
   - 作工細緻
   - 無異味
   - 使用起來很舒適
   - 出貨速度快

3. 商品類型：拉力帶
Fees (1M calls):  4330
Tokens:  283
-------------------
Gemini_API
1. **情緒分析：** positive
2. **優點：**
    * 作工細緻
    * 無異味
    * 使用舒適
3. **商品分類：** 拉力帶
Fees (1M calls): 31.0
Tokens: 158
-------------------
Mistral_API
open-mistral-7b
1. 情緒分析: 這是一則正面回覆，表達買家對這款拉力帶的滿意和感激之情。
2. 針對物品本身的分析:  Optimizes workmanship, odorless, comfortable to use.
3. 根據內容分類: This text describes a resistance band.
Fees (1M calls): 76.5
Tokens: 306
open-mixtral-8x7b
1. 情緒分析：positive
2. 優點：
	* 作工細緻
	* 無異味
	* 使用很舒適
	* 超讚的出貨速度
3. 分類：拉力帶
Fees (1M calls): 201.59999999999997
Tokens: 288
mistral-small-latest
1. 情緒分析：positive
2. 優點：
   -

In [79]:
# Transform
prompt = """
請將「」中的文字翻譯成英文，並以 JSON 格式回傳 {{ origin: , translation:  }}

「{}」
"""

contents = [
    """
    床前明月光，疑是地上霜
    """,
    """
    人在江湖身不由己
    """
]

show_llm_result(prompt, contents)


Prompt: 
請將「」中的文字翻譯成英文，並以 JSON 格式回傳 { origin: , translation:  }

「
    床前明月光，疑是地上霜
    」

OpenAI_API
gpt-3.5-turbo
{
    "origin": "床前明月光，疑是地上霜",
    "translation": "The bright moonlight in front of the bed, I suspect it is frost on the ground."
}
Fees (1M calls):  107.0
Tokens:  118
gpt-4-turbo-preview
```json
{
  "origin": "床前明月光，疑是地上霜",
  "translation": "Before my bed, the moonlight glows, like frost upon the ground"
}
```
Fees (1M calls):  2170
Tokens:  119
-------------------
Gemini_API
```json
{
  "origin": "床前明月光，疑是地上霜",
  "translation": "The bright moon is shining in front of my bed, I suspect that it is frost on the ground."
}
```
Fees (1M calls): 24.375
Tokens: 93
-------------------
Mistral_API
open-mistral-7b
{
  "origin": "There is a bright moonlight shining in front of the bed, it seems that the ground is frostbitten",
  "translation": "Moonlight over the Ruined Porch\nBy an Old Pond\nThe moon shines on the broken edge of an old porch,\nA frosty night."
}

Note: The Engli

In [84]:
# Expand
prompt = """
你是專業的客服，你需要針對「」中的回饋，產生禮貌且專業的回覆
如果語氣是正面，則感謝用戶的留言，如果語氣是負面則道歉並請用戶聯繫真人克服
請從用戶回饋中摘要特定細節，並以專業且用心的口吻撰寫
總字數不超過 200 字

「{}」
"""

contents = [
    """
    這週三中午跟同事一起訂購了火腿炒飯加味增湯體驗很好，週五下班點了2份鮭魚炒飯，炒飯粒粒分明清爽美味，不油不膩，值得推薦！
    味增湯料多好喝，重點是魚都沒有魚刺，讓我這個雙寶爸下班都要準備孩子晚餐多了一個口袋名單！
    """,
    """
    我先不管頭殼有沒有丼
    認真評論
    你們蘿蔔絲真的是超級難吃，你們有自己吃過嗎
    整個魚腥味超級重，很像那種生魚出水，然後把蘿蔔絲拿進那腥水裡面攪一攪再放進便當裡，又濕又臭。相比之下醋飯好太多
    第一次吃到這麼不新鮮的蘿蔔絲 太扯了吧
    """
]

show_llm_result(prompt, contents)


Prompt: 
你是專業的客服，你需要針對「」中的回饋，產生禮貌且專業的回覆
如果語氣是正面，則感謝用戶的留言，如果語氣是負面則道歉並請用戶聯繫真人克服
請從用戶回饋中摘要特定細節，並以專業且用心的口吻撰寫
總字數不超過 200 字

「
    這週三中午跟同事一起訂購了火腿炒飯加味增湯體驗很好，週五下班點了2份鮭魚炒飯，炒飯粒粒分明清爽美味，不油不膩，值得推薦！
    味增湯料多好喝，重點是魚都沒有魚刺，讓我這個雙寶爸下班都要準備孩子晚餐多了一個口袋名單！
    」

OpenAI_API
gpt-3.5-turbo
謝謝您寶貴的回饋！我們很高興得知您對火腿炒飯和味增湯的體驗都非常滿意，也感謝您對鮭魚炒飯的肯定！我們一直努力提供新鮮、美味且健康的餐點給我們的顧客，希望能繼續為您帶來愉快的用餐體驗。如果您有任何其他需求或建議，歡迎隨時與我們聯繫。期待您下次再光臨！祝您和孩子們都有美味愉快的用餐時光！
Fees (1M calls):  508.5
Tokens:  CompletionUsage(completion_tokens=224, prompt_tokens=345, total_tokens=569)
gpt-4-turbo-preview
親愛的顧客，

非常感謝您抽出寶貴時間來提供這份溫暖的回饋。我們很高興聽到您與同事在週三中午的火腿炒飯加味增湯體驗讓您滿意，同時，您在週五下班選擇的2份鮭魚炒飯也得到了您的肯定和推薦。了解到炒飯的粒粒分明與清爽口感，以及味增湯料豐富、鮭魚無刺讓您喜愛，讓我們感到無比開心。

我們將繼續努力，提供品質卓越的美食，希望繼續成為您與家庭愉悅用餐的選擇。若您有任何建議或需要協助，歡迎隨時聯絡我們。

再次感謝您的支持與認可！

最誠摯的，
[您的公司團隊]
Fees (1M calls):  13260
Tokens:  CompletionUsage(completion_tokens=327, prompt_tokens=345, total_tokens=672)
-------------------
Gemini_API
親愛的顧客，

感謝您提供的熱情回饋，我們很高興您對我們的火腿炒飯、鮭魚炒飯和味增湯感到滿意。您的表揚對我們團隊意義重大，我們將持續努力提供美味且優質的餐點。

我們很高興得知我們的味增湯